In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd
import datetime
from datetime import date
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler, normalize
from sklearn import metrics
from sklearn.mixture import GaussianMixture
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import warnings
warnings.filterwarnings('ignore')
data=pd.read_csv('../input/customer-personality-analysis/marketing_campaign.csv',header=0,sep='\t')

In [3]:
data.shape

In [4]:
data.head()

In [5]:
!pip install dataprep

In [6]:
from dataprep.eda import plot, plot_correlation, create_report, plot_missing
plot(data)

In [7]:
data['Age']=2014-data['Year_Birth']
data['Spending']=data['MntWines']+data['MntFruits']+data['MntMeatProducts']+data['MntFishProducts']+data['MntSweetProducts']+data['MntGoldProds']


In [8]:
#Storing the latest data.
last_date=date(2014,6,29)
#Converting the Object datatype to datetime64 and storing in the column "Seniority".
data['Seniority']=pd.to_datetime(data['Dt_Customer'], dayfirst=True,format = '%d-%m-%Y')
#How many month old the customer is from the latest date with the company is stored in "Seniority Column".
data['Seniority'] = pd.to_numeric(data['Seniority'].dt.date.apply(lambda x: (last_date - x)).dt.days, downcast='integer')/30


In [9]:
#Renaming the columns with appropriate names
data=data.rename(columns={'NumWebPurchases':"Web","NumCatalogPurchases":"Catalog","NumStorePurchases":"Store"})

In [10]:
data['Marital_Status'].unique()

In [11]:
#Categorizing the marital status into 'Alone' and 'In couple'
data['Marital_Status']=data['Marital_Status'].replace({'Divorced':'Alone','Single':'Alone','Married':'In couple','Together':'In couple','Absurd':'Alone','Widow':'Alone','YOLO':'Alone'})

In [12]:
data['Marital_Status'].unique()

In [13]:
data['Education'].unique()

In [14]:
#Converting education into 'undergraduate' and 'postgraduate'
data['Education']=data['Education'].replace({'Graduation':'Postgraduate','PhD':'Postgraduate','Master':'Postgraduate','Basic':'Undergraduate','2n Cycle':'Undergraduate'})

In [15]:
data['Education'].unique()

In [16]:
data['Children']=data['Kidhome']+data['Teenhome']

In [17]:
data.head()

In [18]:
data['Has_child']=np.where(data.Children>0,'Has child','No child')

In [19]:
#change the column values of children
data['Children'].replace({3:"3 children",2:"2 children",1:"1 child",0:"No child"},inplace=True)

In [20]:
data.head()

In [21]:
#Renaming more columns with relevant names
data=data.rename(columns={'MntWines':'Wines','MntFruits':'Fruits','MntMeatProducts':'Meat','MntFishProducts':'Fish','MntSweetProducts':'Sweets','MntGoldProds':'Gold'})

In [22]:
data=data[['Age','Education','Marital_Status','Income','Spending','Seniority','Has_child','Children','Wines','Fruits','Meat','Fish','Sweets','Gold']]
data.head()

In [23]:
#Checking null values
data.isnull().sum()

In [24]:
#Dropped rows with Income as NaN
data=data.dropna(subset=['Income'])

In [25]:
data.isnull().sum()

In [26]:
fig, ax = plt.subplots(figsize=(16, 6))

sns.boxplot(data['Income'])

In [27]:
data['Income'].describe()

In [28]:
#Calculate IQR=Q3-Q1
68522-35303

In [29]:
#Upper bound value = Q3+1.5 x IQR
68522+(1.5*33219)

In [30]:
#Lower Bound Value = Q1-1.5 x IQR
35303-(1.5*33219)

In [31]:
data.shape

In [32]:
#There are 8 records which falls beyond upper bound outlier, hence delete them.
data.drop(data[data['Income']>=118350.5].index,inplace=True)

In [33]:
#8 rows deleted
data.shape

In [34]:
fig,axes=plt.subplots(figsize=(16,8))
sns.boxplot(data['Income'])

# Clustering the data distrubution #

We will create 4 categories :-
1. Old customers with high income and high spending nature.
2. New customer with below average income and low spending nature
3. new customers with high income  and high spending nature
4. Old customers with below-average income and low spending nature

In [35]:
#Normalize the data and cluster
scaler=StandardScaler()
dataset_temp=data[['Income','Seniority','Spending']]
scaled_data=pd.DataFrame(scaler.fit_transform(dataset_temp))

In [36]:
scaled_data.head()

In [37]:
X=normalize(scaled_data,norm='l2')

In [38]:
gmm=GaussianMixture(n_components=4,covariance_type='spherical',max_iter=2000,random_state=5).fit(X)
labels=gmm.predict(X)
dataset_temp['Cluster']=labels
dataset_temp=dataset_temp.replace({0:'Stars',1:'Need attention',2:'High potential',3:'Leaky bucket'})


In [39]:
data=data.merge(dataset_temp.Cluster,left_index=True,right_index=True)

In [40]:
data.head()

In [41]:
pd.options.display.float_format="{:.0f}".format

In [42]:
summary=data[['Income','Spending','Seniority','Cluster']]
summary.set_index('Cluster',inplace=True)
summary=summary.groupby('Cluster').describe().transpose()
summary

In [43]:
#Plot this data to see the customer clustering
PLOT=go.Figure()
for C in list(data.Cluster.unique()):
    PLOT.add_trace(go.Scatter3d(x=data[data.Cluster==C]['Income'],
                                y=data[data.Cluster==C]['Seniority'],
                                z=data[data.Cluster==C]['Spending'],
                                mode='markers',marker_size=6,marker_line_width=1,
                                name=str(C)))
PLOT.update_traces(hovertemplate='Income: %{x} <br>Seniority: %{y} <br>Spending: %{z}')

PLOT.update_layout(width=800,height=800,autosize=True,showlegend=True,
                   scene=dict(xaxis=dict(title='Income',titlefont_color='black'),
                              yaxis=dict(title='Seniority',titlefont_color='black'),
                              zaxis=dict(title='Spending',titlefont_color='black')),
                   font=dict(family="Gilroy",color='black',size=12))

# Data Preparation for customer personality Analysis
# 

In [44]:
#Creating Age Segment
cut_labels_Age=['Young','Adult','Mature','Senior']
cut_bins=[0,30,45,65,120]
data['Age_group']=pd.cut(data['Age'],bins=cut_bins,labels=cut_labels_Age)

In [45]:
data.head()

In [46]:
#Creating Income Segment
cut_labels_Income=['Low income','Low to medium income','Medium to high income','High income']
data['Income_group']=pd.qcut(data['Income'],q=4,labels=cut_labels_Income)

In [47]:
data.head()

In [48]:
#Creating seniority segment
cut_labels_Seniority=['New customers','Discovering customers','Experienced customers','Old customers']
data['Seniority_group']=pd.qcut(data['Seniority'],q=4,labels=cut_labels_Seniority)

In [49]:
data=data.drop(columns=['Age','Income','Seniority'])

In [50]:
data.head()

# More segmentation based on spending habbit #

In [51]:
cut_labels=['Low consumer','Frequent consumer','Biggest consumer']
data['Wines_segment']=pd.qcut(data['Wines'][data['Wines']>0],q=[0,.25,.75,1],labels=cut_labels).astype("object")


In [52]:
data['Fruits_segment']=pd.qcut(data['Fruits'][data['Fruits']>0],q=[0,.25,.75,1],labels=cut_labels).astype("object")
data['Meat_segment']=pd.qcut(data['Meat'][data['Meat']>0],q=[0,.25,.75,1],labels=cut_labels).astype("object")
data['Fish_segment']=pd.qcut(data['Fish'][data['Fish']>0],q=[0,.25,.75,1],labels=cut_labels).astype("object")
data['Sweets_segment']=pd.qcut(data['Sweets'][data['Sweets']>0],q=[0,.25,.75,1],labels=cut_labels).astype("object")
data['Gold_segment']=pd.qcut(data['Gold'][data['Gold']>0],q=[0,.25,.75,1],labels=cut_labels).astype("object")
data.replace(np.nan,'Non consumer',inplace=True)
data.drop(columns=['Spending','Wines','Fruits','Meat','Fish','Sweets','Gold'],inplace=True)
data=data.astype(object)

In [53]:
data.head()

In [54]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.max_colwidth',999)
pd.options.display.float_format="{:.3f}".format
association=data.copy()
df=pd.get_dummies(association)
min_support=0.08
max_len=10
frequent_items=apriori(df,use_colnames=True,min_support=min_support,max_len=max_len+1)
rules=association_rules(frequent_items,metric='lift',min_threshold=1)
rules.head()

In [56]:
product='Wines'
segment='Biggest consumer'
target='{\'%s_segment_%s\'}' %(product,segment)
results_personal_care=rules[rules['consequents'].astype(str).str.contains(target,na=False)].sort_values(by='confidence',ascending=False)
results_personal_care.head()
# print(target)

# Insights

**Biggest Customters of Wines are**

* Customer with an average income of 50000 or more.
* Customer with an average total spend 1200 or more.
* Customers registered with the company for 21 months approx.
* Cutomers with graduate degree.
* also Customers who are heavy consumer of meat products.